In [ ]:
import pandas as pd
import numpy as np
import sklearn
from math import pi
from tqdm.auto import tqdm

In [ ]:
# Read from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install cudf-cu12

In [ ]:
# import cudf

In [ ]:
# cudf.DataFrame.from_pandas(df)

In [ ]:
N_WEEKS_IN_YEARS = 52.18
EVENT_TYPES = ['Religious', 'National', 'Cultural', 'Sporting']

In [ ]:
# # Function to reduce memory usage
# def reduce_mem_usage(df):
#     # start_mem = df.memory_usage().sum() / 1024**2
#     # print(f'Memory usage of dataframe is {start_mem:.2f} MB')

#     for col in df.columns:
#         col_type = df[col].dtype

#         if col_type != object:
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type)[:3] == 'int':
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64)
#             else:
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
#                 elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float64)
#         else:
#             df[col] = df[col].astype('category')

#     # end_mem = df.memory_usage().sum() / 1024**2
#     # print(f'Memory usage after optimization is: {end_mem:.2f} MB')
#     # print(f'Decreased by {(start_mem - end_mem) / start_mem * 100:.1f}%')

#     return df

In [ ]:
def reduce_mem_usage(df):
    """
    Reduces memory usage of a pandas DataFrame by optimizing data types.
    Handles numeric, categorical, timestamp, and string data intelligently.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame to optimize

    Returns:
    --------
    pandas.DataFrame
        The memory-optimized DataFrame
    """
    import numpy as np
    import pandas as pd
    import re

    # start_mem = df.memory_usage().sum() / 1024**2
    # print(f'Memory usage of dataframe is {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        # Handle timestamp data that's already recognized
        if pd.api.types.is_datetime64_any_dtype(col_type):
            # Timestamps are already efficiently stored
            continue

        # Handle numeric data (int and float)
        elif pd.api.types.is_numeric_dtype(col_type):
            c_min = df[col].min()
            c_max = df[col].max()

            # Handle integer data
            if pd.api.types.is_integer_dtype(col_type):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)

            # Handle float data
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

        # Handle boolean data
        elif pd.api.types.is_bool_dtype(col_type):
            df[col] = df[col].astype(bool)

        # Handle string and object data
        elif pd.api.types.is_object_dtype(col_type) or pd.api.types.is_string_dtype(col_type):
            # Skip timestamp detection for columns where we already know they're not timestamps
            if df[col].dtype == 'object' and len(df[col].dropna()) > 0:
                # Get the first non-null value to examine
                first_val = df[col].dropna().iloc[0]

                # Skip datetime conversion if it's not a string or if it's numeric-like
                if not isinstance(first_val, str) or first_val.isdigit():
                    pass
                # Skip datetime conversion for very short strings that can't be dates
                elif isinstance(first_val, str) and len(first_val) < 6:
                    pass
                else:
                    # Only attempt conversion if it looks like a date
                    # Common date patterns as regex
                    date_patterns = [
                        r'\d{4}[-/]\d{1,2}[-/]\d{1,2}',  # YYYY-MM-DD or YYYY/MM/DD
                        r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}',  # DD-MM-YYYY or MM-DD-YYYY
                        r'\d{1,2}\s+[A-Za-z]{3,9}\s+\d{2,4}',  # DD Month YYYY
                        r'[A-Za-z]{3,9}\s+\d{1,2},?\s+\d{2,4}'  # Month DD, YYYY
                    ]

                    # Check if first value matches any date pattern
                    is_date_like = False
                    for pattern in date_patterns:
                        if re.search(pattern, str(first_val)):
                            is_date_like = True
                            break

                    if is_date_like:
                        try:
                            # Try common explicit formats instead of using automatic detection
                            formats_to_try = [
                                '%Y-%m-%d',
                                '%Y/%m/%d',
                                '%d-%m-%Y',
                                '%d/%m/%Y',
                                '%Y-%m-%d %H:%M:%S',
                                '%Y/%m/%d %H:%M:%S',
                                '%d-%m-%Y %H:%M:%S',
                                '%d/%m/%Y %H:%M:%S',
                                '%b %d %Y',
                                '%B %d %Y',
                                '%d %b %Y',
                                '%d %B %Y'
                            ]

                            converted = False
                            for fmt in formats_to_try:
                                try:
                                    # Try to convert using explicit format
                                    temp_series = pd.to_datetime(df[col].head(10), format=fmt, errors='raise')
                                    # If successful, apply to the whole column
                                    df[col] = pd.to_datetime(df[col], format=fmt, errors='coerce')
                                    converted = True
                                    break
                                except ValueError:
                                    continue

                            # Last resort: if none of the formats work but it still looks like dates,
                            # skip datetime conversion to avoid the warning
                            if not converted:
                                # Instead of using automatic detection, leave as is
                                pass
                        except:
                            pass

            # If it's not a timestamp, check for categorical potential
            if not pd.api.types.is_datetime64_any_dtype(df[col].dtype):
                n_unique = df[col].nunique()
                n_total = len(df)

                # Convert to category only if it has a reasonable number of unique values
                if n_unique / n_total < 0.5:  # Less than 50% unique values
                    df[col] = df[col].astype('category')

        # Handle categorical data
        elif pd.api.types.is_categorical_dtype(col_type):
            pass  # Already optimized

    # end_mem = df.memory_usage().sum() / 1024**2
    # print(f'Memory usage after optimization is: {end_mem:.2f} MB')
    # print(f'Decreased by {(start_mem - end_mem) / start_mem * 100:.1f}%')

    return df

# Load Data

## Sales Train Evaluation

In [ ]:
sales_train_evaluation_chunks = pd.read_csv('/content/drive/My Drive/time_series/competition_data/sales_train_evaluation.csv', chunksize=1000)

In [ ]:
# sales_train_evaluation = pd.read_csv('/content/drive/My Drive/time_series/competition_data/sales_train_evaluation.csv')

In [ ]:
sales_train_evaluation.loc[:, "d_1908": "d_1913"].head()

,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,1,1,3,0,1,1
1,0,1,0,0,0,0
2,1,1,0,1,1,1
3,1,0,1,3,7,2
4,1,1,2,2,2,4


In [ ]:
sales_train_evaluation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


**d_1914** through **d_1941** is the prediction period. For submission is **d_1942** through **d_1969**.

In [ ]:
sales_train_evaluation.shape

(30490, 1947)

## Sales Train Validation

In [ ]:
sales_train_validation_chunks = pd.read_csv('/content/drive/My Drive/time_series/competition_data/sales_train_validation.csv', chunksize=1000)

In [ ]:
sales_train_validation = cudf.read_csv('/content/drive/My Drive/time_series/competition_data/sales_train_validation.csv')

In [ ]:
sales_train_validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [ ]:
sales_train_validation.shape

(30490, 1919)

**d_1886** through **d_1913** is the prediction period.  For submission is **d_1914** through **d_1941**.

## Calendar

In [ ]:
calendar = pd.read_csv('/content/drive/My Drive/time_series/competition_data/calendar.csv')

In [ ]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


### Handle week of year

In [ ]:
# calendar["wm_yr_wk_len"] = calendar["wm_yr_wk"]#apply(lambda x: len(str(x)))
# #calendar["wm_yr_wk_len"].value_counts()

In [ ]:
calendar["wm_yr_wk_0"] = calendar["wm_yr_wk"].astype('str').str.get(0)
calendar["wm_yr_wk_0"].value_counts()

,count
wm_yr_wk_0,
1,1969


In [ ]:
calendar["week_of_year"] = calendar["wm_yr_wk"].astype('str').str[3:].astype(int)

In [ ]:
calendar["week_of_year"].tail()

,week_of_year
1964,20
1965,20
1966,20
1967,21
1968,21


In [ ]:
# Sine and cosine transformation for week number
calendar['week_sin'] = np.sin(2 * np.pi * calendar['week_of_year']/N_WEEKS_IN_YEARS)
calendar['week_cos'] = np.cos(2 * np.pi * calendar['week_of_year']/N_WEEKS_IN_YEARS)

In [ ]:
calendar['week_cos']

,week_cos
0,0.992759
1,0.992759
2,0.992759
3,0.992759
4,0.992759
...,...
1964,-0.742957
1965,-0.742957
1966,-0.742957
1967,-0.817980


### Examine events

In [ ]:
# I am checking the number of nulls to examine the null relation in events of type 1 vs events of type 2

In [ ]:
calendar.loc[:, "event_name_1": "event_type_2"].isnull().sum()

,0
event_name_1,1807
event_type_1,1807
event_name_2,1964
event_type_2,1964


In [ ]:
# Making sure that if I have events in type 2, I must have events in type 1
calendar[calendar["event_name_2"].notnull()]["event_type_1"].isnull().sum()

np.int64(0)

In [ ]:
calendar[calendar["event_name_2"].notnull()].head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,wm_yr_wk_0,week_of_year,week_sin,week_cos
85,2011-04-24,11113,Sunday,2,4,2011,d_86,OrthodoxEaster,Religious,Easter,Cultural,0,0,0,1,13,0.999985,0.005419
827,2013-05-05,11315,Sunday,2,5,2013,d_828,OrthodoxEaster,Religious,Cinco De Mayo,Cultural,1,1,1,1,15,0.972419,-0.233240
1177,2014-04-20,11412,Sunday,2,4,2014,d_1178,Easter,Cultural,OrthodoxEaster,Religious,0,0,0,1,12,0.992094,0.125500
1233,2014-06-15,11420,Sunday,2,6,2014,d_1234,NBAFinalsEnd,Sporting,Father's day,Cultural,0,1,1,1,20,0.669339,-0.742957
1968,2016-06-19,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,1,21,0.575247,-0.817980


In [ ]:
calendar["event_type_1"].value_counts()

,count
event_type_1,
Religious,55
National,52
Cultural,37
Sporting,18


In [ ]:
calendar["event_type_1"].value_counts().index

Index(['Religious', 'National', 'Cultural', 'Sporting'], dtype='object', name='event_type_1')

In [ ]:
calendar["event_type_2"].value_counts()

,count
event_type_2,
Cultural,4
Religious,1


In [ ]:
calendar["event_name_1"].nunique()

30

### Add time to next event

In [ ]:
def create_days_until_next_event_columns(df):
    """
    Creates columns showing days until the next event by category.

    Parameters:
    -----------
    df : pandas DataFrame
        DataFrame with date and event_type columns where most values are NaN

    Returns:
    --------
    DataFrame with additional columns for days until next event by category
    """
    # Make a copy of the input dataframe to avoid modifying the original
    result_df = df.copy()

    # Ensure date is in datetime format
    result_df['date'] = pd.to_datetime(result_df['date'])

    # Sort by date
    result_df = result_df.sort_values('date').reset_index(drop=True)

    # For each event type
    for event_type in EVENT_TYPES:
        # Create column name
        col_name = f'days_until_{event_type}'

        # Initialize column with NaN
        result_df[col_name] = np.nan

        # Find dates with this event type
        event_indices_1 = result_df[result_df['event_type_1'] == event_type].index
        event_indices_2 = result_df[result_df['event_type_2'] == event_type].index

        # Combine indices and get unique dates
        event_indices = np.union1d(event_indices_1, event_indices_2)
        event_dates = result_df.loc[event_indices, 'date'].values

        # For each date in our dataframe
        for i, row in result_df.iterrows():
            current_date = row['date']

            # Find future event dates (including current date)
            future_event_dates = event_dates[event_dates >= current_date]

            if len(future_event_dates) > 0:
                # If event occurs on the same day
                if current_date == future_event_dates[0]:
                    days_until = 0
                else:
                    # Calculate days difference
                    next_event_date = pd.to_datetime(future_event_dates[0])
                    days_until = (next_event_date - current_date).days

                # Update the result dataframe
                result_df.loc[i, col_name] = days_until

    return result_df

In [ ]:
calendar = create_days_until_next_event_columns(calendar)

In [ ]:
calendar.head(10)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,...,snap_TX,snap_WI,wm_yr_wk_0,week_of_year,week_sin,week_cos,days_until_Religious,days_until_National,days_until_Cultural,days_until_Sporting
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,...,0,0,1,1,0.120123,0.992759,39.0,23.0,16.0,8.0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,...,0,0,1,1,0.120123,0.992759,38.0,22.0,15.0,7.0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,...,0,0,1,1,0.120123,0.992759,37.0,21.0,14.0,6.0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,...,1,0,1,1,0.120123,0.992759,36.0,20.0,13.0,5.0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,...,0,1,1,1,0.120123,0.992759,35.0,19.0,12.0,4.0
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,...,1,1,1,1,0.120123,0.992759,34.0,18.0,11.0,3.0
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,...,0,0,1,1,0.120123,0.992759,33.0,17.0,10.0,2.0
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,...,1,1,1,2,0.238506,0.971141,32.0,16.0,9.0,1.0
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,...,1,1,1,2,0.238506,0.971141,31.0,15.0,8.0,0.0
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,...,1,0,1,2,0.238506,0.971141,30.0,14.0,7.0,113.0


In [ ]:
def create_event_info_columns(df):
    """
    Creates columns showing:
    1. Days until the next event by category
    2. Name of the next event (if it's less than 10 days away)

    Parameters:
    -----------
    df : pandas DataFrame
        DataFrame with date, event_type, and event_name columns

    Returns:
    --------
    DataFrame with additional columns for days until next event and event names
    """
    # Make a copy to avoid modifying the original
    result_df = df.copy()

    # Ensure date is in datetime format
    result_df['date'] = pd.to_datetime(result_df['date'])

    # Sort by date
    result_df = result_df.sort_values('date').reset_index(drop=True)

    # List of event types
    event_types = ['Religious', 'National', 'Cultural', 'Sporting']

    # For each event type
    for event_type in event_types:
        # Create column names
        days_col_name = f'days_until_{event_type}'
        name_col_name = f'next_{event_type}_event_name'

        # Initialize columns with NaN
        result_df[days_col_name] = np.nan
        result_df[name_col_name] = np.nan

        # Find rows with this event type and get their indices
        event_indices_1 = result_df[result_df['event_type_1'] == event_type].index
        event_indices_2 = result_df[result_df['event_type_2'] == event_type].index

        # Combine indices and get unique values
        event_indices = np.union1d(event_indices_1, event_indices_2)

        # Create a dictionary mapping event dates to event names
        event_date_to_name = {}

        for idx in event_indices:
            date = result_df.loc[idx, 'date']
            # Check which event type column has this event
            if result_df.loc[idx, 'event_type_1'] == event_type:
                event_name = result_df.loc[idx, 'event_name_1']
            else:
                event_name = result_df.loc[idx, 'event_name_2']

            event_date_to_name[date] = event_name

        # Get array of event dates
        event_dates = np.array(list(event_date_to_name.keys()))

        # For each date in our dataframe
        for i, row in result_df.iterrows():
            current_date = row['date']

            # Find future event dates (including current date)
            future_dates_mask = event_dates >= current_date
            future_event_dates = event_dates[future_dates_mask]

            if len(future_event_dates) > 0:
                # Get the next event date
                next_event_date = future_event_dates[0]

                # Calculate days until next event - fixed to use days attribute
                days_until = (next_event_date - current_date).days

                # Store days until next event
                result_df.loc[i, days_col_name] = days_until

                # Store event name only if it's less than 10 days away
                if days_until < 10:
                    result_df.loc[i, name_col_name] = event_date_to_name[next_event_date]

    return result_df

In [ ]:
calendar = create_event_info_columns(calendar)

<ipython-input-26-115ad6277242>:81: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'LentStart' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result_df.loc[i, name_col_name] = event_date_to_name[next_event_date]
<ipython-input-26-115ad6277242>:81: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'PresidentsDay' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result_df.loc[i, name_col_name] = event_date_to_name[next_event_date]
<ipython-input-26-115ad6277242>:81: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'ValentinesDay' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result_df.loc[i, name_col_name] = event_date_to_nam

In [ ]:
calendar.head(20)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,...,week_sin,week_cos,days_until_Religious,days_until_National,days_until_Cultural,days_until_Sporting,next_Religious_event_name,next_National_event_name,next_Cultural_event_name,next_Sporting_event_name
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,...,0.120123,0.992759,39.0,23.0,16.0,8.0,NaN,NaN,NaN,SuperBowl
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,...,0.120123,0.992759,38.0,22.0,15.0,7.0,NaN,NaN,NaN,SuperBowl
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,...,0.120123,0.992759,37.0,21.0,14.0,6.0,NaN,NaN,NaN,SuperBowl
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,...,0.120123,0.992759,36.0,20.0,13.0,5.0,NaN,NaN,NaN,SuperBowl
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,...,0.120123,0.992759,35.0,19.0,12.0,4.0,NaN,NaN,NaN,SuperBowl
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,...,0.120123,0.992759,34.0,18.0,11.0,3.0,NaN,NaN,NaN,SuperBowl
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,...,0.120123,0.992759,33.0,17.0,10.0,2.0,NaN,NaN,NaN,SuperBowl
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,...,0.238506,0.971141,32.0,16.0,9.0,1.0,NaN,NaN,ValentinesDay,SuperBowl
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,...,0.238506,0.971141,31.0,15.0,8.0,0.0,NaN,NaN,ValentinesDay,SuperBowl
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,...,0.238506,0.971141,30.0,14.0,7.0,113.0,NaN,NaN,ValentinesDay,NaN


In [ ]:
def split_multiple_events(df):
    """
    Splits rows with two events into separate rows, each with one event.

    Parameters:
    -----------
    df : pandas DataFrame
        DataFrame with event_name_1, event_type_1, event_name_2, event_type_2 columns

    Returns:
    --------
    DataFrame with single event_name, event_type columns and duplicated rows for multiple events
    """
    # Make a copy to avoid modifying the original
    result_df = df.copy()

    # Create new columns for the unified event data
    result_df['event_name'] = result_df['event_name_1']
    result_df['event_type'] = result_df['event_type_1']

    # Identify rows with a second event (where event_type_2 is not NaN)
    second_event_mask = result_df['event_type_2'].notna()
    rows_with_second_event = result_df[second_event_mask].copy()

    if not rows_with_second_event.empty:
        # For the second set of events, update the event columns
        rows_with_second_event['event_name'] = rows_with_second_event['event_name_2']
        rows_with_second_event['event_type'] = rows_with_second_event['event_type_2']

        # Concatenate the original dataframe with the new rows for second events
        result_df = pd.concat([result_df, rows_with_second_event], ignore_index=True)


    # Sort by date and other columns for a clean result
    result_df = result_df.sort_values(['id', 'date', 'event_type']).reset_index(drop=True)

    return result_df

In [ ]:
# calendar_dup = split_multiple_events(calendar).reset_index()

In [ ]:
# Check results
calendar_dup[calendar_dup["event_name_2"].notnull()].head().sort_values(["date"]).head(10)

,index,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,...,days_until_Religious,days_until_National,days_until_Cultural,days_until_Sporting,next_Religious_event_name,next_National_event_name,next_Cultural_event_name,next_Sporting_event_name,event_name,event_type
85,85,2011-04-24,11113,Sunday,2,4,2011,d_86,OrthodoxEaster,Religious,...,0.0,36.0,0.0,37.0,OrthodoxEaster,NaN,Easter,NaN,Easter,Cultural
86,86,2011-04-24,11113,Sunday,2,4,2011,d_86,OrthodoxEaster,Religious,...,0.0,36.0,0.0,37.0,OrthodoxEaster,NaN,Easter,NaN,OrthodoxEaster,Religious
828,828,2013-05-05,11315,Sunday,2,5,2013,d_828,OrthodoxEaster,Religious,...,0.0,22.0,0.0,32.0,OrthodoxEaster,NaN,Cinco De Mayo,NaN,Cinco De Mayo,Cultural
829,829,2013-05-05,11315,Sunday,2,5,2013,d_828,OrthodoxEaster,Religious,...,0.0,22.0,0.0,32.0,OrthodoxEaster,NaN,Cinco De Mayo,NaN,OrthodoxEaster,Religious
1179,1179,2014-04-20,11412,Sunday,2,4,2014,d_1178,Easter,Cultural,...,0.0,36.0,0.0,46.0,OrthodoxEaster,NaN,Easter,NaN,Easter,Cultural


In [ ]:
calendar_dup["date"].max()

Timestamp('2016-06-19 00:00:00')

### Reduce memory

In [ ]:
# calendar_dup = reduce_mem_usage(calendar_dup)

In [ ]:
calendar = reduce_mem_usage(calendar)

In [ ]:
calendar.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,...,week_sin,week_cos,days_until_Religious,days_until_National,days_until_Cultural,days_until_Sporting,next_Religious_event_name,next_National_event_name,next_Cultural_event_name,next_Sporting_event_name
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,...,0.120117,0.992676,39.0,23.0,16.0,8.0,NaN,NaN,NaN,SuperBowl
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,...,0.120117,0.992676,38.0,22.0,15.0,7.0,NaN,NaN,NaN,SuperBowl
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,...,0.120117,0.992676,37.0,21.0,14.0,6.0,NaN,NaN,NaN,SuperBowl
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,...,0.120117,0.992676,36.0,20.0,13.0,5.0,NaN,NaN,NaN,SuperBowl
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,...,0.120117,0.992676,35.0,19.0,12.0,4.0,NaN,NaN,NaN,SuperBowl


## Sell Prices

In [ ]:
prices = pd.read_csv('/content/drive/My Drive/time_series/competition_data/sell_prices.csv')

In [ ]:
prices.shape

(6841121, 4)

In [ ]:
prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [ ]:
# Check is the prices are different for different stores
prices[(prices["item_id"]=="HOBBIES_1_001") & (prices["wm_yr_wk"]==11326)]

,store_id,item_id,wm_yr_wk,sell_price
1,CA_1,HOBBIES_1_001,11326,9.58
1335807,CA_3,HOBBIES_1_001,11326,8.26
2029798,CA_4,HOBBIES_1_001,11326,9.58
2708823,TX_1,HOBBIES_1_001,11326,8.26
4800945,WI_1,HOBBIES_1_001,11326,9.58


Not necessarily

In [ ]:
# Add an index column (the price evolution over time compared to the original price of the item in the store)

In [ ]:
# First, let's sort the dataframe by wm_yr_wk to ensure chronological order
prices = prices.sort_values(by=['store_id', 'item_id', 'wm_yr_wk'])

# Create a group key for each store_id and item_id combination
prices['store_item_key'] = prices['store_id'] + '_' + prices['item_id']

# Find the first week for each store_item_key
first_appearances = prices.groupby('store_item_key')['wm_yr_wk'].transform('min')

In [ ]:
first_appearances.head()

,wm_yr_wk
368746,11101
368747,11101
368748,11101
368749,11101
368750,11101


In [ ]:
# Create a boolean mask for rows that represent the first appearance
is_first_appearance = (prices['wm_yr_wk'] == first_appearances)

# Get the original price for each store_item_key
original_prices = prices[is_first_appearance].set_index('store_item_key')['sell_price']

In [ ]:
original_prices.head()

,sell_price
store_item_key,
CA_1_FOODS_1_001,2.00
CA_1_FOODS_1_002,7.88
CA_1_FOODS_1_003,2.88
CA_1_FOODS_1_004,1.78
CA_1_FOODS_1_005,2.94


In [ ]:
# Map these original prices back to all rows
prices['original_price'] = prices['store_item_key'].map(original_prices)
# Calculate the price index
prices['price_index'] = prices['sell_price'] / prices['original_price']

In [ ]:
prices.tail()

,store_id,item_id,wm_yr_wk,sell_price,store_item_key,original_price,price_index
6512550,WI_3,HOUSEHOLD_2_516,11617,5.94,WI_3_HOUSEHOLD_2_516,5.94,1.0
6512551,WI_3,HOUSEHOLD_2_516,11618,5.94,WI_3_HOUSEHOLD_2_516,5.94,1.0
6512552,WI_3,HOUSEHOLD_2_516,11619,5.94,WI_3_HOUSEHOLD_2_516,5.94,1.0
6512553,WI_3,HOUSEHOLD_2_516,11620,5.94,WI_3_HOUSEHOLD_2_516,5.94,1.0
6512554,WI_3,HOUSEHOLD_2_516,11621,5.94,WI_3_HOUSEHOLD_2_516,5.94,1.0


In [ ]:
# Now create a column with the ratio between the current price to the previous week price
prices = prices.sort_values(by=['store_id', 'item_id', 'wm_yr_wk'])
prices['prev_week_price'] = prices.groupby('store_item_key')['sell_price'].shift(1)
prices['week_to_week_ratio'] = prices['sell_price'] / prices['prev_week_price']

In [ ]:
prices.head()

,store_id,item_id,wm_yr_wk,sell_price,store_item_key,original_price,price_index,prev_week_price,week_to_week_ratio
368746,CA_1,FOODS_1_001,11101,2.0,CA_1_FOODS_1_001,2.0,1.0,NaN,NaN
368747,CA_1,FOODS_1_001,11102,2.0,CA_1_FOODS_1_001,2.0,1.0,2.0,1.0
368748,CA_1,FOODS_1_001,11103,2.0,CA_1_FOODS_1_001,2.0,1.0,2.0,1.0
368749,CA_1,FOODS_1_001,11104,2.0,CA_1_FOODS_1_001,2.0,1.0,2.0,1.0
368750,CA_1,FOODS_1_001,11105,2.0,CA_1_FOODS_1_001,2.0,1.0,2.0,1.0


In [ ]:
prices['week_to_week_ratio'] = prices['week_to_week_ratio'].fillna(1.0)

In [ ]:
# Now create a column with the ratio of the current week to the average price of the preceding 4 weeks
prices = prices.sort_values(by=['store_id', 'item_id', 'wm_yr_wk'])
# Calculate the mean of the preceding 4 weeks' prices
# Use a rolling window with a minimum of 1 observation
prices['mean_prev_4_weeks'] = prices.groupby('store_item_key')['sell_price'].transform(
    lambda x: x.shift(1).rolling(window=4, min_periods=4).mean()
)

In [ ]:
prices

,store_id,item_id,wm_yr_wk,sell_price,store_item_key,original_price,price_index,prev_week_price,week_to_week_ratio,mean_prev_4_weeks
368746,CA_1,FOODS_1_001,11101,2.00,CA_1_FOODS_1_001,2.00,1.0,NaN,1.0,NaN
368747,CA_1,FOODS_1_001,11102,2.00,CA_1_FOODS_1_001,2.00,1.0,2.00,1.0,NaN
368748,CA_1,FOODS_1_001,11103,2.00,CA_1_FOODS_1_001,2.00,1.0,2.00,1.0,NaN
368749,CA_1,FOODS_1_001,11104,2.00,CA_1_FOODS_1_001,2.00,1.0,2.00,1.0,NaN
368750,CA_1,FOODS_1_001,11105,2.00,CA_1_FOODS_1_001,2.00,1.0,2.00,1.0,2.00
...,...,...,...,...,...,...,...,...,...,...
6512550,WI_3,HOUSEHOLD_2_516,11617,5.94,WI_3_HOUSEHOLD_2_516,5.94,1.0,5.94,1.0,5.94
6512551,WI_3,HOUSEHOLD_2_516,11618,5.94,WI_3_HOUSEHOLD_2_516,5.94,1.0,5.94,1.0,5.94
6512552,WI_3,HOUSEHOLD_2_516,11619,5.94,WI_3_HOUSEHOLD_2_516,5.94,1.0,5.94,1.0,5.94
6512553,WI_3,HOUSEHOLD_2_516,11620,5.94,WI_3_HOUSEHOLD_2_516,5.94,1.0,5.94,1.0,5.94


In [ ]:
# Calculate the ratio of current price to the mean of the preceding 4 weeks
prices['ratio_to_4wk_mean'] = prices['sell_price'] / prices['mean_prev_4_weeks']
prices['ratio_to_4wk_mean'] = prices['ratio_to_4wk_mean'].fillna(1.0)

In [ ]:
prices.head()

,store_id,item_id,wm_yr_wk,sell_price,store_item_key,original_price,price_index,prev_week_price,week_to_week_ratio,mean_prev_4_weeks,ratio_to_4wk_mean
368746,CA_1,FOODS_1_001,11101,2.0,CA_1_FOODS_1_001,2.0,1.0,NaN,1.0,NaN,1.0
368747,CA_1,FOODS_1_001,11102,2.0,CA_1_FOODS_1_001,2.0,1.0,2.0,1.0,NaN,1.0
368748,CA_1,FOODS_1_001,11103,2.0,CA_1_FOODS_1_001,2.0,1.0,2.0,1.0,NaN,1.0
368749,CA_1,FOODS_1_001,11104,2.0,CA_1_FOODS_1_001,2.0,1.0,2.0,1.0,NaN,1.0
368750,CA_1,FOODS_1_001,11105,2.0,CA_1_FOODS_1_001,2.0,1.0,2.0,1.0,2.0,1.0


In [ ]:
prices = reduce_mem_usage(prices[["store_id", "item_id", "wm_yr_wk", "sell_price",
                                  "week_to_week_ratio", "ratio_to_4wk_mean"]])

<ipython-input-5-82a3d67ac4e4>:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('category')
<ipython-input-5-82a3d67ac4e4>:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('category')
<ipython-input-5-82a3d67ac4e4>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

# Preprocessing

In [ ]:
calendar_dup.head()

,index,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,...,days_until_Religious,days_until_National,days_until_Cultural,days_until_Sporting,next_Religious_event_name,next_National_event_name,next_Cultural_event_name,next_Sporting_event_name,event_name,event_type
0,0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,...,39.0,23.0,16.0,8.0,NaN,NaN,NaN,SuperBowl,NaN,NaN
1,1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,...,38.0,22.0,15.0,7.0,NaN,NaN,NaN,SuperBowl,NaN,NaN
2,2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,...,37.0,21.0,14.0,6.0,NaN,NaN,NaN,SuperBowl,NaN,NaN
3,3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,...,36.0,20.0,13.0,5.0,NaN,NaN,NaN,SuperBowl,NaN,NaN
4,4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,...,35.0,19.0,12.0,4.0,NaN,NaN,NaN,SuperBowl,NaN,NaN


In [ ]:
calendar_dup["date"] = pd.to_datetime(calendar_dup["date"])

In [ ]:
calendar_dup["date"].max()

Timestamp('2016-06-19 00:00:00')

In [ ]:
c# alendar_dup = calendar_dup.rename({"d": "index_day"}, axis=1)

In [ ]:
calendar.columns

Index(['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'wm_yr_wk_0', 'week_of_year',
       'week_sin', 'week_cos', 'days_until_Religious', 'days_until_National',
       'days_until_Cultural', 'days_until_Sporting',
       'next_Religious_event_name', 'next_National_event_name',
       'next_Cultural_event_name', 'next_Sporting_event_name'],
      dtype='object')

In [ ]:
calendar_cols = ["date", "wm_yr_wk", "weekday", "wday", "month", "index_day",
                 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
                 "snap_CA", "snap_TX", "snap_WI",
                 "week_sin", "week_cos",
                 'days_until_Religious', 'days_until_National', 'days_until_Cultural', 'days_until_Sporting',
                 'next_Religious_event_name', 'next_National_event_name',
                 'next_Cultural_event_name', 'next_Sporting_event_name'
                 ]

In [ ]:
def process_chunk(chunk, calendar):
    chunk = \
    chunk.set_index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']) \
    .stack() \
    .rename("sales").reset_index()

    chunk = chunk.rename({"level_6": "index_day"}, axis=1)
    calendar = calendar.rename({"d": "index_day"}, axis=1)

    # Merge sales_train_evaluation_long with calendar_cudf
    chunk = chunk.merge(calendar[calendar_cols], on="index_day", how="left")

    # Merge with prices
    chunk = chunk.merge(prices, on=["store_id", "item_id", "wm_yr_wk"], how="left")

    return chunk

In [ ]:
calendar.columns

Index(['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'wm_yr_wk_0', 'week_of_year',
       'week_sin', 'week_cos', 'days_until_Religious', 'days_until_National',
       'days_until_Cultural', 'days_until_Sporting',
       'next_Religious_event_name', 'next_National_event_name',
       'next_Cultural_event_name', 'next_Sporting_event_name'],
      dtype='object')

In [ ]:
# calender_dup.columns

In [ ]:
# Process and merge with calendar
results = []
for chunk in tqdm(sales_train_validation_chunks): #sales_train_evaluation_chunks / sales_train_validation_chunks
    # Move chunk to GPU
    chunk = reduce_mem_usage(chunk)
    # Process on GPU
    processed = process_chunk(chunk, calendar)
    # Add to results (can keep on GPU or move back to CPU)
    results.append(processed)

0it [00:00, ?it/s]

In [ ]:
sales_train_validation = pd.concat(results, ignore_index=True) # sales_train_evaluation/sales_train_validation

In [ ]:
sales_train_validation["id"].nunique()

30490

In [ ]:
def consecutive_counter(series):
    """
    Converts consecutive 1s in a series to an incrementing counter (1, 2, 3...)
    Each new streak of 1s starts again from 1.

    Examples:
    [0,0,1,1,1,1,0,0] becomes [0,0,1,2,3,4,0,0]
    [0,1,1,0,0,0,1,0] becomes [0,1,2,0,0,0,1,0]

    Parameters:
    -----------
    series : pandas Series
        Series with 0s and 1s

    Returns:
    --------
    pandas Series with consecutive 1s converted to incrementing counters
    """
    # Create a mask of the 1s
    mask = series == 1

    # Create groups by detecting changes in the mask
    # This assigns a new group number each time the series changes from 0 to 1 or 1 to 0
    groups = mask.ne(mask.shift()).cumsum()

    # For each group, if it contains 1s, create an incrementing counter
    result = pd.Series(0, index=series.index)

    for group_id, group in series.groupby(groups):
        if group.iloc[0] == 1:  # If this group consists of 1s
            # Create incrementing counter for this group
            result.loc[group.index] = np.arange(1, len(group) + 1)

    return result

In [ ]:
def consecutive_counter_optimized(series):
    """
    Efficiently converts consecutive 1s in a series to an incrementing counter (1, 2, 3...)
    Uses vectorized operations for maximum performance.

    Parameters:
    -----------
    series : pandas Series
        Series with 0s and 1s (where 1 represents no sales)

    Returns:
    --------
    pandas Series with consecutive 1s converted to incrementing counters
    """
    # If series is empty or contains no 1s, return it as is
    if len(series) == 0 or not (series == 1).any():
        return pd.Series(0, index=series.index)

    # Create groups by detecting changes in the series
    # This creates a new group ID each time the value changes
    groups = series.ne(series.shift()).cumsum()

    # Create a DataFrame to store the original series and group IDs
    df = pd.DataFrame({'value': series, 'group': groups})

    # For each group with 1s, create cumulative counts
    # First, identify which groups have 1s
    groups_with_ones = df[df['value'] == 1]['group'].unique()

    # Initialize the result with zeros
    result = pd.Series(0, index=series.index)

    # Compute cumulative counts for groups with 1s
    if len(groups_with_ones) > 0:
        mask = df['group'].isin(groups_with_ones) & (df['value'] == 1)
        # Use cumcount to efficiently assign incrementing values
        df.loc[mask, 'counter'] = df.loc[mask].groupby('group').cumcount() + 1
        # Copy the counters to the result
        result.loc[mask.index[mask]] = df.loc[mask, 'counter']

    return result

In [ ]:
def calculate_sales_metrics_vectorized(df):
    """
    Fully vectorized implementation to calculate sales metrics:
    1. Average sales for previous 7, 14, and 28 days
    2. Standard deviation of sales for previous 7, 14, and 28 days
    3. Number of days since last sale (correctly implemented)

    Parameters:
    -----------
    df : pandas DataFrame
        DataFrame with columns: id, date, store_id, sales

    Returns:
    --------
    DataFrame with additional metrics columns
    """
    # Make a copy to avoid modifying the original
    result_df = df.copy()

    # Ensure date is in datetime format
    result_df['date'] = pd.to_datetime(result_df['date'])

    # Sort by id (product-store), then by date
    result_df = result_df.sort_values(['id', 'date']).reset_index(drop=True)

    # List of window sizes to calculate metrics for
    windows = [7, 14, 28]

    # Create a flag for days with sales
    result_df['has_sales'] = result_df['sales'] > 0

    # Process each window size for rolling statistics
    for window in windows:
        # Use rolling window operations by group
        avg_sales = result_df.groupby('id')['sales'].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean().shift(1)
        )
        std_sales = result_df.groupby('id')['sales'].transform(
            lambda x: x.rolling(window=window, min_periods=1).std().shift(1)
        )

        # Store results
        result_df[f'avg_sales_{window}d'] = avg_sales
        result_df[f'std_sales_{window}d'] = std_sales




    return result_df

In [ ]:
sales_train_validation = calculate_sales_metrics_vectorized(sales_train_validation) # sales_train_validation/sales_train_evaluation

In [ ]:
# Vectorized days_without_sales calculation
# Create a shift of sales
sales_train_validation['shifted_sales'] = sales_train_validation.groupby('id')['sales'].shift(1)

In [ ]:
# Assign 0 if lag has sales
sales_train_validation['no_sales_in_lag'] = 0
sales_train_validation.loc[sales_train_validation['shifted_sales']==0,'no_sales_in_lag'] = 1

In [ ]:
sales_train_validation.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,id,item_id,dept_id,cat_id,store_id,state_id,index_day,sales,date,wm_yr_wk,...,ratio_to_4wk_mean,has_sales,avg_sales_7d,std_sales_7d,avg_sales_14d,std_sales_14d,avg_sales_28d,std_sales_28d,shifted_sales,no_sales_in_lag
0,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3,2011-01-29,11101,...,1.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_2,0,2011-01-30,11101,...,1.0,False,3.0,NaN,3.0,NaN,3.0,NaN,3.0,0
2,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_3,0,2011-01-31,11101,...,1.0,False,1.5,2.121320,1.5,2.121320,1.5,2.121320,0.0,1
3,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_4,1,2011-02-01,11101,...,1.0,True,1.0,1.732051,1.0,1.732051,1.0,1.732051,0.0,1
4,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_5,4,2011-02-02,11101,...,1.0,True,1.0,1.414214,1.0,1.414214,1.0,1.414214,1.0,0


In [ ]:
# Convert 1s sequences to 1,2..
sales_train_validation['days_without_sales'] = sales_train_validation.groupby('id')['no_sales_in_lag'].transform(consecutive_counter_optimized)

In [ ]:
sales_train_validation[["date", "sales", "days_without_sales"]].head(10)

,date,sales,days_without_sales
0,2011-01-29,3,0
1,2011-01-30,0,0
2,2011-01-31,0,1
3,2011-02-01,1,2
4,2011-02-02,4,0
5,2011-02-03,2,0
6,2011-02-04,0,0
7,2011-02-05,2,1
8,2011-02-06,0,0
9,2011-02-07,0,1


In [ ]:
def create_lag_lead_features(df):
    """
    Efficiently creates two sets of features:
    1. Lag sales features (past sales): 1-28 days back
    2. Lead sales features (future sales): 1-27 days ahead

    Parameters:
    -----------
    df : pandas DataFrame
        DataFrame with columns: id, date, sales
        Where id is a unique product-store identifier

    Returns:
    --------
    DataFrame with additional lag and lead columns
    """
    # Make a copy to avoid modifying the original
    result_df = df.copy()

    # Ensure date is in datetime format
    result_df['date'] = pd.to_datetime(result_df['date'])

    # Sort by id (product-store), then by date
    result_df = result_df.sort_values(['id', 'date'])

    # Create lag features (past sales) in a vectorized way
    for i in tqdm(range(1, 29)):  # 1 to 28 days back
        col_name = f'lag_sales_{i}'
        result_df[col_name] = result_df.groupby('id')['sales'].shift(i)

    # Create lead features (future sales) in a vectorized way
    for i in tqdm(range(1, 28)):  # 1 to 27 days ahead
        col_name = f'y_lead_{i}'
        result_df[col_name] = result_df.groupby('id')['sales'].shift(-i)

    return result_df

In [ ]:
sales_train_validation = create_lag_lead_features(sales_train_validation) # sales_train_validation/sales_train_evaluation

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

In [ ]:
# sales_train_validation.shape

(58327370, 25)

In [ ]:
sales_train_validation.iloc[:50,-40:-20]

,lag_sales_16,lag_sales_17,lag_sales_18,lag_sales_19,lag_sales_20,lag_sales_21,lag_sales_22,lag_sales_23,lag_sales_24,lag_sales_25,lag_sales_26,lag_sales_27,lag_sales_28,y_lead_1,y_lead_2,y_lead_3,y_lead_4,y_lead_5,y_lead_6,y_lead_7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,4.0,2.0,0.0,2.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,4.0,2.0,0.0,2.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,2.0,0.0,2.0,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0,0.0,2.0,0.0,0.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,0.0,0.0,0.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,0.0,0.0,0.0,0.0,3.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,3.0,1.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,3.0,1.0,3.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.0,1.0,3.0,0.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,3.0,1.0,3.0,0.0,2.0


In [ ]:
# sales_train_validation.isnull().sum()

,0
id,0
item_id,0
dept_id,0
cat_id,0
store_id,0
state_id,0
index_day,0
sales,0
date,0
wm_yr_wk,0


In [ ]:
sales_train_validation[["week_to_week_ratio", "ratio_to_4wk_mean"]] = \
sales_train_validation[["week_to_week_ratio", "ratio_to_4wk_mean"]].fillna(1.0) # sales_train_evaluation/sales_train_validation

In [ ]:
sales_train_validation = split_multiple_events(sales_train_validation).reset_index()

In [ ]:
for col in sales_train_validation.columns:
    print(col)

index
id
item_id
dept_id
cat_id
store_id
state_id
index_day
sales
date
wm_yr_wk
weekday
wday
month
event_name_1
event_type_1
event_name_2
event_type_2
snap_CA
snap_TX
snap_WI
week_sin
week_cos
days_until_Religious
days_until_National
days_until_Cultural
days_until_Sporting
next_Religious_event_name
next_National_event_name
next_Cultural_event_name
next_Sporting_event_name
sell_price
week_to_week_ratio
ratio_to_4wk_mean
has_sales
avg_sales_7d
std_sales_7d
avg_sales_14d
std_sales_14d
avg_sales_28d
std_sales_28d
shifted_sales
no_sales_in_lag
days_without_sales
lag_sales_1
lag_sales_2
lag_sales_3
lag_sales_4
lag_sales_5
lag_sales_6
lag_sales_7
lag_sales_8
lag_sales_9
lag_sales_10
lag_sales_11
lag_sales_12
lag_sales_13
lag_sales_14
lag_sales_15
lag_sales_16
lag_sales_17
lag_sales_18
lag_sales_19
lag_sales_20
lag_sales_21
lag_sales_22
lag_sales_23
lag_sales_24
lag_sales_25
lag_sales_26
lag_sales_27
lag_sales_28
y_lead_1
y_lead_2
y_lead_3
y_lead_4
y_lead_5
y_lead_6
y_lead_7
y_lead_8
y_lead_9


In [ ]:
sales_train_validation.shape

(58449330, 101)

In [ ]:
sales_train_validation[:4].T[30:120]

,0,1,2,3
lag_sales_17,NaN,NaN,NaN,NaN
lag_sales_18,NaN,NaN,NaN,NaN
lag_sales_19,NaN,NaN,NaN,NaN
lag_sales_20,NaN,NaN,NaN,NaN
lag_sales_21,NaN,NaN,NaN,NaN
lag_sales_22,NaN,NaN,NaN,NaN
lag_sales_23,NaN,NaN,NaN,NaN
lag_sales_24,NaN,NaN,NaN,NaN
lag_sales_25,NaN,NaN,NaN,NaN
lag_sales_26,NaN,NaN,NaN,NaN


In [ ]:
# sales_train_evaluation/sales_train_validation
sales_train_validation.to_parquet('/content/drive/My Drive/time_series/competition_data/wip/sales_train_validation_processed5.parquet', index=False)

# General Data Overview

- daily (days of week) / Weekly / monthly seasonality
- event effect on sales. Try to estinguish between evets that impact hte same day and those that affect in advance
- price effect on sales